In [ ]:
from lameg.util import ctf_fif_spm_conversion
import os
import glob

In [ ]:

def upload_file(rawdata_path, sub, ses, condition):
    """
    Search and upload the raw datachunck

    :param rawdata_path: datapath
    :param sub: subjects id
    :param ses: session id 
    :param condition: experimental condition (block-wise)
    """

    sub_meg_path = os.path.join(rawdata_path, "sub-" + sub , "ses-0" + ses , "meg/sub-"+ sub + "_ses-0" + ses + "_task-" + condition + "_acq-")
    folders_acq = glob.glob(sub_meg_path + "*_meg.ds", recursive=True)
    list_folders = sorted(folders_acq, key=lambda x: int(x.split('acq-')[1].split('_')[0]))
    
    return list_folders

In [ ]:
rawdata_path = ""

SUB = ['105', '107', '109', '113', '114',
           '115', '117','118','119', '123', 
           '124', '126','127','129', '130',
           '131','132','133','134','135', 
           '136', '137','138', '141','142',
           '143','144','145']

ses = "1"
condition = "V100V100"
mxf_option = "no_mxf"
triggers = ["right_stim", "left_stim"]

for s, sub in enumerate(SUB): 
    
    list_folders = upload_file(rawdata_path, sub, ses, condition)
    mxf_path_dir = os.path.join(list_folders[-1], f"mxf_{mxf_option}_data", "mxf_sess_data")

    output_spm_conversion_dir = os.path.join(mxf_path_dir, "mxf_sess_data", "spm_convert_epo")
    os.makedirs(output_spm_conversion_dir, exist_ok=True)

    for trigger in triggers:
                
        epochs_file_path = os.path.join(mxf_path_dir, f"sub-{sub}_ses-0{ses}_task-{condition}_meg_{mxf_option}_{trigger}_nrg_ERF_epo.fif" )

        if os.path.exists(epochs_file_path): 
            print("Found epochs for SPM conversion")
            print(epochs_file_path)
        else: 
            print(f"Epochs not found - check {epochs_file_path}")

        print(f"Currently processing sub {sub} ses {ses} condition {condition} epochs {trigger}")

        last_ctf_rec_path = list_folders[1]
        if os.path.exists(last_ctf_rec_path): 
            print("Found raw ctf folder")
        else: print("SRaw ctf folder not found")

        record_num = int(last_ctf_rec_path.split('acq-')[1].split('_')[0])
        print(f"Using the raw CTF file of {sub} ses {ses} condition {condition} recording sess {record_num} to extract info")
    
        ctf_fif_spm_conversion(
            epochs_file_path, last_ctf_rec_path, output_spm_conversion_dir, 
            True
        )

        print(f"Conversion finished for {epochs_file_path}")
        